In [1]:
import nltk
import numpy as np
import random
import string
import bs4 as bs
import urllib.request
import re

In [2]:
# Creating the corpus
raw_html = urllib.request.urlopen('https://en.wikipedia.org/wiki/Tennis')
raw_html = raw_html.read()
print(raw_html)

b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Tennis - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"ba313040-9840-4961-a99a-9a6de356755e","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Tennis","wgTitle":"Tennis","wgCurRevisionId":1015447106,"wgRevisionId":1015447106,"wgArticleId":29773,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1: Julian\xe2\x80\x93Gregorian uncertainty","Webarchive template wayback links","CS1 maint: multiple names: authors list","Articles with short description","Short description is dif

In [3]:
article_html = bs.BeautifulSoup(raw_html, 'lxml')
print(article_html)

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Tennis - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"ba313040-9840-4961-a99a-9a6de356755e","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Tennis","wgTitle":"Tennis","wgCurRevisionId":1015447106,"wgRevisionId":1015447106,"wgArticleId":29773,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1: Julian–Gregorian uncertainty","Webarchive template wayback links","CS1 maint: multiple names: authors list","Articles with short description","Short description is different from Wikida

In [4]:
article_paragraphs = article_html.find_all('p')
print(article_paragraphs)

[<p class="mw-empty-elt">
</p>, <p><b>Tennis</b> is a <a class="mw-redirect" href="/wiki/Racket_sport" title="Racket sport">racket sport</a> that can be played individually against a single opponent (<a href="/wiki/Types_of_tennis_match#Singles" title="Types of tennis match">singles</a>) or between two teams of two players each (<a href="/wiki/Types_of_tennis_match#Doubles" title="Types of tennis match">doubles</a>). Each player uses a <a class="mw-redirect" href="/wiki/Tennis_racket" title="Tennis racket">tennis racket</a> that is strung with cord to strike a hollow rubber <a href="/wiki/Tennis_ball" title="Tennis ball">ball</a> covered with felt over or around a net and into the opponent's <a href="/wiki/Tennis_court" title="Tennis court">court</a>. The object of the game is to maneuver the ball in such a way that the opponent is not able to play a valid return. The player who is unable to return the ball will not gain a point, while the opposite player will.
</p>, <p>Tennis is an <a

In [5]:
article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()
print(article_text)


tennis is a racket sport that can be played individually against a single opponent (singles) or between two teams of two players each (doubles). each player uses a tennis racket that is strung with cord to strike a hollow rubber ball covered with felt over or around a net and into the opponent's court. the object of the game is to maneuver the ball in such a way that the opponent is not able to play a valid return. the player who is unable to return the ball will not gain a point, while the opposite player will.
tennis is an olympic sport and is played at all levels of society and at all ages. the sport can be played by anyone who can hold a racket, including wheelchair users. the modern game of tennis originated in birmingham, england, in the late 19th century as lawn tennis.[1] it had close connections both to various field (lawn) games such as croquet and bowls as well as to the older racket sport today called real tennis. during most of the 19th century, in fact, the term tennis r

In [7]:
# Text processing and helper function
article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)
article_text = re.sub(r'\s+', ' ', article_text)
article_sentences = nltk.sent_tokenize(article_text)
article_words = nltk.word_tokenize(article_text)

In [8]:
# Remove the punctuation from the user input text and lemmatize the text
wnlemmatizer = nltk.stem.WordNetLemmatizer()

def perform_lemmatization(tokens):
    return [wnlemmatizer.lemmatize(token) for token in tokens]

punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_processed_text(document):
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

In [9]:
# Responding to greetings
greeting_inputs = ("hey", "good morning", "good evening", "morning", "evening", "hi", "whatsup")
greeting_responses = ["hey", "hey hows you?", "*nods*", "hello, how you doing", "hello", "Welcome, I am good and you"]

def generate_greeting_response(greeting):
    for token in greeting.split():
        if token.lower() in greeting_inputs:
            return random.choice(greeting_responses)

In [10]:
# Responding to user queries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def generate_response(user_input):
    tennisrobo_response = ''
    article_sentences.append(user_input)

    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')
    all_word_vectors = word_vectorizer.fit_transform(article_sentences)
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)
    similar_sentence_number = similar_vector_values.argsort()[0][-2]

    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0:
        tennisrobo_response = tennisrobo_response + "I am sorry, I could not understand you"
        return tennisrobo_response
    else:
        tennisrobo_response = tennisrobo_response + article_sentences[similar_sentence_number]
        return tennisrobo_response

In [11]:
# Convert all the sentences in the corpus along with the input sequence into their corresponding vector form
word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')
all_word_vectors = word_vectorizer.fit_transform(article_sentences)

C:\Users\REEHA\anaconda3\envs\ibm\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [12]:
# Cosine similarity between the last item in the all_word_vectors list
similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)
similar_sentence_number = similar_vector_values.argsort()[0][-2]

In [16]:
# Chatting with the chatbot
continue_dialogue = True
print("Hello, I am your friend TennisRobo. You can ask me any question regarding tennis:")
while(continue_dialogue == True):
    human_text = input()
    human_text = human_text.lower()
    if human_text != 'bye':
        if human_text == 'thanks' or human_text == 'thank you very much' or human_text == 'thank you':
            continue_dialogue = False
            print("TennisRobo: Most welcome")
        else:
            if generate_greeting_response(human_text) != None:
                print("TennisRobo: " + generate_greeting_response(human_text))
            else:
                print("TennisRobo: ", end="")
                print(generate_response(human_text))
                article_sentences.remove(human_text)
    else:
        continue_dialogue = False
        print("TennisRobo: Good bye and take care of yourself...")

Hello, I am your friend TennisRobo. You can ask me any question regarding tennis:
Roger Federer
TennisRobo: roger federer is considered by many observers to have the most "complete" game in modern tennis.
exit
TennisRobo: I am sorry, I could not understand you
bye
TennisRobo: Good bye and take care of yourself...
